In [1344]:
#Imports
import PIL.Image as Image
import math
import cairo
import numpy as np

In [1345]:
def to_pil(surface: cairo.ImageSurface) -> Image:
    format = surface.get_format()
    size = (surface.get_width(), surface.get_height())
    stride = surface.get_stride()

    with surface.get_data() as memory:
        if format == cairo.Format.RGB24:
            return Image.frombuffer(
                "RGB", size, memory.tobytes(),
                'raw', "BGRX", stride)
        elif format == cairo.Format.ARGB32:
            return Image.frombuffer(
                "RGBA", size, memory.tobytes(),
                'raw', "BGRa", stride)
        else:
            raise NotImplementedError(repr(format))

In [1346]:
#canvas
WIDTH, HEIGHT = 1000, 1000
POINTSIZE = 3
surface = cairo.ImageSurface(cairo.FORMAT_ARGB32, WIDTH, HEIGHT)
ctx = cairo.Context(surface)
ctx.set_line_width(6)

In [1347]:
def fillBlack():
    ctx.set_source_rgba(0.0, 0.0, 0.0, 1) 
    ctx.rectangle(0, 0, WIDTH, HEIGHT)
    ctx.fill()
fillBlack()

In [1348]:
def toPolar3D(x,y,z):
    r = math.sqrt(x**2 + y**2 + z**2)
    theta = 0 if r == 0 else math.acos(z/r)
    phi = 0 if x == 0 and y == 0 else (-1 if y < 0 else 1)*math.acos(x/(math.sqrt(x**2 + y**2)))
    return r, theta, phi

In [1349]:
def toCartesian3D(r, theta, phi):
    x = r * math.sin(theta)* math.cos(phi)
    y = r * math.sin(theta)* math.sin(phi)
    z = r * math.cos(theta)
    return x, y, z

In [1350]:
def project3dTo2d(x,y,z):
    cameraDistance = 500
    angleX = math.atan(x/(z + cameraDistance))
    newX = cameraDistance*(math.sin(angleX)/(math.cos(angleX)))

    angleY = math.atan(y/(z + cameraDistance))
    newY = cameraDistance*(math.sin(angleY)/(math.cos(angleY)))
    return newX, newY

In [1351]:
def project3dTo2dX(x,y,z):
    cameraDistance = 1000
    angleY = math.atan(y/(x + cameraDistance))
    newY = cameraDistance*(math.sin(angleY)/(math.cos(angleY)))

    angleZ = math.atan(z/(x + cameraDistance))
    newZ = cameraDistance*(math.sin(angleZ)/(math.cos(angleZ)))
    return newY, newZ

In [1352]:
def buildCube(radius, height):
    top0 = toPolar3D(radius, radius,height)
    top1 = toPolar3D(-radius, radius,height)
    top2 = toPolar3D(radius, -radius,height)
    top3 = toPolar3D(-radius, -radius,height)

    bot0 = toPolar3D(radius, radius,-height)
    bot1 = toPolar3D(-radius, radius,-height)
    bot2 = toPolar3D(radius, -radius,-height)
    bot3 = toPolar3D(-radius, -radius,-height)

    return top0,top1,top2,top3,bot0,bot1,bot2,bot3

In [1353]:
def buildFigure(radius, height):
    centre0 = toPolar3D(0, 0,height)
    centre1 = toPolar3D(0, 0,-height)

    top0 = toPolar3D(radius, 0,height)
    top1 = toPolar3D(-radius, 0,height)
    top2 = toPolar3D(0, radius,height)
    top3 = toPolar3D(0, -radius,height)

    bot0 = toPolar3D(radius, 0,-height)
    bot1 = toPolar3D(-radius, 0,-height)
    bot2 = toPolar3D(0, radius,-height)
    bot3 = toPolar3D(0, -radius,-height)
    return centre0,centre1,top0,top1,top2,top3,bot0,bot1,bot2,bot3

In [1354]:
#THIS SECTION TO ADD POINTS

#points
pointList = []
#x, y, z
#static
point0 = toPolar3D(0,0,0)

pointList.append(point0)

#FIGURES
figPoints0 = []
for point in buildFigure(50, 70):
    figPoints0.append(point)

figPoints1 = []
for point in buildFigure(80, 100):
    figPoints1.append(point)

figPoints2 = []
for point in buildFigure(110, 130):
    figPoints2.append(point)

figPoints3 = []
for point in buildFigure(140, 160):
    figPoints3.append(point)

figPoints4 = []
for point in buildFigure(170, 190):
    figPoints4.append(point)


In [1355]:
def movePoint1(point):
    return [point[0], point[1]+0.1, point[2]]

In [1356]:
def movePoint2(point):
    return [point[0], point[1], point[2]+(math.pi/30)]

In [1357]:
def drawPoint(point):
    x,y,z = toCartesian3D(point[0], point[1], point[2])
    x,y = project3dTo2dX(x,y,z)
    x = x + WIDTH//2
    y = y + HEIGHT//2
    ctx.arc(x,y,POINTSIZE,0,math.pi*2)
    ctx.fill()

In [1358]:
def drawLine(point0, point1):
    x0,y0,z0 = toCartesian3D(point0[0], point0[1], point0[2])
    x0,y0 = project3dTo2dX(x0,y0,z0)

    x1,y1,z1 = toCartesian3D(point1[0], point1[1], point1[2])
    x1,y1 = project3dTo2dX(x1,y1,z1)

    ctx.move_to(WIDTH//2 +x0,HEIGHT//2 +y0)
    ctx.line_to(WIDTH//2 +x1,HEIGHT//2 +y1)
    ctx.stroke()

In [1359]:
def drawCube(cubePoints):
    drawLine(cubePoints[0], cubePoints[1])
    drawLine(cubePoints[2], cubePoints[3])
    drawLine(cubePoints[4], cubePoints[5])
    drawLine(cubePoints[6], cubePoints[7])

    drawLine(cubePoints[1], cubePoints[5])
    drawLine(cubePoints[0], cubePoints[4])
    drawLine(cubePoints[2], cubePoints[6])
    drawLine(cubePoints[3], cubePoints[7])

    drawLine(cubePoints[0], cubePoints[2])
    drawLine(cubePoints[1], cubePoints[3])
    drawLine(cubePoints[4], cubePoints[6])
    drawLine(cubePoints[5], cubePoints[7])

In [1360]:
def drawFig(figPoints):
    drawLine(figPoints[0], figPoints[1])

    drawLine(figPoints[0], figPoints[2])
    drawLine(figPoints[0], figPoints[3])
    drawLine(figPoints[0], figPoints[4])
    drawLine(figPoints[0], figPoints[5])

    drawLine(figPoints[1], figPoints[6])
    drawLine(figPoints[1], figPoints[7])
    drawLine(figPoints[1], figPoints[8])
    drawLine(figPoints[1], figPoints[9])

    drawLine(figPoints[2], figPoints[6])
    drawLine(figPoints[3], figPoints[7])
    drawLine(figPoints[4], figPoints[8])
    drawLine(figPoints[5], figPoints[9])

In [1361]:
#DrawLoop
imageList = []
for iterator in range(100):
    fillBlack()
    ctx.set_source_rgba(1.0, 1.0, 1.0, 1) 
    
    for i in range(len(pointList)):
        drawPoint(pointList[i])
    
    #fig
    for i in range(len(figPoints0)):
        if iterator >= 0:
            if any(not element[2] > math.pi for element in figPoints0[:]):
                figPoints0[i] = movePoint2(figPoints0[i])
        drawPoint(figPoints0[i])
    drawFig(figPoints0)
    for i in range(len(figPoints1)):
        if iterator >= 3:
            if any(not element[2] > math.pi for element in figPoints1[:]):
                figPoints1[i] = movePoint2(figPoints1[i])
        drawPoint(figPoints1[i])
    drawFig(figPoints1)
    for i in range(len(figPoints2)):
        if iterator >= 6:
            if any(not element[2] > math.pi for element in figPoints2[:]):
                figPoints2[i] = movePoint2(figPoints2[i])
        drawPoint(figPoints2[i])
    drawFig(figPoints2)
    for i in range(len(figPoints3)):
        if iterator >= 9:
            if any(not element[2] > math.pi for element in figPoints3[:]):
                figPoints3[i] = movePoint2(figPoints3[i])
        drawPoint(figPoints3[i])
    drawFig(figPoints3)
    for i in range(len(figPoints4)):
        if iterator >= 12:
            if any(not element[2] > math.pi for element in figPoints4[:]):
                figPoints4[i] = movePoint2(figPoints4[i])
        drawPoint(figPoints4[i])
    drawFig(figPoints4)


    imageList.append(to_pil(surface))


In [1362]:
#Save as png
surface.write_to_png("images/example.png")

In [1363]:
#Save as gif
frame_one = imageList[0]
frame_one.save("images/example.gif", format="GIF", append_images=imageList,
               save_all=True, duration=0, loop=0)